In [1]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.5 MB/s eta 0:00:00a 0:00:01


In [2]:
!git clone --branch baselineCe https://github.com/Graph-Classification-Noisy-Label/hackaton.git

Cloning into 'hackaton'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 81 (delta 1), reused 1 (delta 1), pack-reused 77 (from 1)
Receiving objects: 100% (81/81), 105.83 MiB | 45.98 MiB/s, done.
Resolving deltas: 100% (13/13), done.


In [3]:
%cd hackaton/

/kaggle/working/hackaton


In [4]:
#Codice prof -> Tolto gdown perche da errori
#!gdown --folder https://drive.google.com/drive/folders/1Z-1JkPJ6q4C6jX4brvq1VRbJH5RPUCAk -O datasets
!wget -O data.zip "https://www.dropbox.com/scl/fi/p11t5ppnbjf1d20t8kd51/data.zip?rlkey=4bdwachswmiwqq0airk08isga&st=2sc06543&dl=1"
!unzip data.zip
!rm data.zip

--2025-05-25 20:34:38--  https://www.dropbox.com/scl/fi/p11t5ppnbjf1d20t8kd51/data.zip?rlkey=4bdwachswmiwqq0airk08isga&st=2sc06543&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.18, 2620:100:601f:18::a27d:912
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ucaf640f6264b62e3f7a24fa4796.dl.dropboxusercontent.com/cd/0/inline/CqWdrppIEyboRlj1Iu1O1y_XKL_jlymVq9eEnbaKlNl9agV0MwNA7fWSZ1iToG5w_n2A0bpk7EYpx7GfDxz7wiGLL0E3j69nkyAmUYe7vD4u_sRl8ROM5MlbmvU33Z5Z1Xt1UkAiaPmEho1v1oha2CGJ/file?dl=1# [following]
--2025-05-25 20:34:39--  https://ucaf640f6264b62e3f7a24fa4796.dl.dropboxusercontent.com/cd/0/inline/CqWdrppIEyboRlj1Iu1O1y_XKL_jlymVq9eEnbaKlNl9agV0MwNA7fWSZ1iToG5w_n2A0bpk7EYpx7GfDxz7wiGLL0E3j69nkyAmUYe7vD4u_sRl8ROM5MlbmvU33Z5Z1Xt1UkAiaPmEho1v1oha2CGJ/file?dl=1
Resolving ucaf640f6264b62e3f7a24fa4796.dl.dropboxusercontent.com (ucaf640f6264b62e3f7a24fa4796.dl.dropboxusercontent.com).

In [5]:
import os
import torch
import pandas as pd
import matplotlib.pyplot as plt
import logging
from tqdm import tqdm
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split
# Load utility functions from cloned repository
from src.loadData import GraphDataset
from src.utils import set_seed
from src.models import GNN
import argparse

# Set the random seed
set_seed()

In [6]:
def add_zeros(data):
    data.x = torch.zeros(data.num_nodes, dtype=torch.long)
    return data

In [7]:
from sklearn.metrics import f1_score
import torch

def train(data_loader, model, optimizer, criterion, device, save_checkpoints, checkpoint_path, current_epoch):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    for data in tqdm(data_loader, desc="Iterating training graphs", unit="batch"):
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, data.y)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()

        pred = output.argmax(dim=1)
        correct += (pred == data.y).sum().item()
        total += data.y.size(0)

        # Accumula predizioni e etichette
        all_preds.extend(pred.cpu().numpy())
        all_labels.extend(data.y.cpu().numpy())

    # Calcolo F1 macro
    f1 = f1_score(all_labels, all_preds, average='macro')

    # Salva il modello se richiesto
    if save_checkpoints:
        checkpoint_file = f"{checkpoint_path}_epoch_{current_epoch + 1}.pth"
        torch.save(model.state_dict(), checkpoint_file)
        print(f"Checkpoint saved at {checkpoint_file}")

    accuracy = correct / total
    avg_loss = total_loss / len(data_loader)

    return avg_loss, accuracy, f1

In [8]:
from sklearn.metrics import f1_score
import torch

def evaluate(data_loader, model, device, calculate_accuracy=False):
    model.eval()
    correct = 0
    total = 0
    predictions = []
    targets = []
    total_loss = 0
    criterion = torch.nn.CrossEntropyLoss()
    
    with torch.no_grad():
        for data in tqdm(data_loader, desc="Iterating eval graphs", unit="batch"):
            data = data.to(device)
            output = model(data)
            pred = output.argmax(dim=1)

            # Raccogli predizioni e label
            predictions.extend(pred.cpu().numpy())
            
            if calculate_accuracy:
                targets.extend(data.y.cpu().numpy())
                correct += (pred == data.y).sum().item()
                total += data.y.size(0)
                total_loss += criterion(output, data.y).item()
    
    if calculate_accuracy:
        # Calcolo F1 macro
        f1 = f1_score(targets, predictions, average='macro')

    if calculate_accuracy:
        accuracy = correct / total
        avg_loss = total_loss / len(data_loader)
        return avg_loss, accuracy, f1

    return predictions

In [9]:
def save_predictions(predictions, test_path):
    script_dir = os.getcwd() 
    submission_folder = os.path.join(script_dir, "submission")
    test_dir_name = os.path.basename(os.path.dirname(test_path))
    
    os.makedirs(submission_folder, exist_ok=True)
    
    output_csv_path = os.path.join(submission_folder, f"testset_{test_dir_name}.csv")
    
    test_graph_ids = list(range(len(predictions)))
    output_df = pd.DataFrame({
        "id": test_graph_ids,
        "pred": predictions
    })
    
    output_df.to_csv(output_csv_path, index=False)
    print(f"Predictions saved to {output_csv_path}")

In [10]:
def plot_training_progress(train_losses, train_accuracies, output_dir):
    epochs = range(1, len(train_losses) + 1)
    plt.figure(figsize=(12, 6))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label="Training Loss", color='blue')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss per Epoch')

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracies, label="Training Accuracy", color='green')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training Accuracy per Epoch')

    # Save plots in the current directory
    os.makedirs(output_dir, exist_ok=True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "training_progress.png"))
    plt.close()

In [11]:
def get_user_input(prompt, default=None, required=False, type_cast=str):

    while True:
        user_input = input(f"{prompt} [{default}]: ")
        
        if user_input == "" and required:
            print("This field is required. Please enter a value.")
            continue
        
        if user_input == "" and default is not None:
            return default
        
        if user_input == "" and not required:
            return None
        
        try:
            return type_cast(user_input)
        except ValueError:
            print(f"Invalid input. Please enter a valid {type_cast.__name__}.")

In [12]:
def get_arguments():
    args = {}
    args['train_path'] = get_user_input("Path to the training dataset (optional)")
    args['test_path'] = get_user_input("Path to the test dataset", required=True)
    args['num_checkpoints'] = get_user_input("Number of checkpoints to save during training", default=5, type_cast=int)
    args['device'] = get_user_input("Which GPU to use if any", default=1, type_cast=int)
    args['gnn'] = get_user_input("GNN type (gin, gin-virtual, gcn, gcn-virtual)", default='gin-virtual')
    args['drop_ratio'] = get_user_input("Dropout ratio", default=0.5, type_cast=float)
    args['num_layer'] = get_user_input("Number of GNN message passing layers", default=5, type_cast=int)
    args['emb_dim'] = get_user_input("Dimensionality of hidden units in GNNs", default=300, type_cast=int)
    args['batch_size'] = get_user_input("Input batch size for training", default=32, type_cast=int)
    args['epochs'] = get_user_input("Number of epochs to train", default=200, type_cast=int)
    args['baseline_mode'] = get_user_input("Baseline mode: 1 (CE), 2 (Noisy CE), 3 (GCE), 4 (SCE)", default=1, type_cast=int)
    args['noise_prob'] = get_user_input("Noise probability p (used if baseline_mode=2)", default=0.2, type_cast=float)

    
    return argparse.Namespace(**args)

In [13]:
def populate_args(args):
    print("Arguments received:")
    for key, value in vars(args).items():
        print(f"{key}: {value}")
args = get_arguments()
populate_args(args)

Path to the training dataset (optional) [None]:  /kaggle/working/hackaton/data/C/train.json.gz
Path to the test dataset [None]:  /kaggle/working/hackaton/data/C/test.json.gz
Number of checkpoints to save during training [5]:  
Which GPU to use if any [1]:  
GNN type (gin, gin-virtual, gcn, gcn-virtual) [gin-virtual]:  
Dropout ratio [0.5]:  
Number of GNN message passing layers [5]:  
Dimensionality of hidden units in GNNs [300]:  
Input batch size for training [32]:  
Number of epochs to train [200]:  
Baseline mode: 1 (CE), 2 (Noisy CE), 3 (GCE), 4 (SCE) [1]:  3
Noise probability p (used if baseline_mode=2) [0.2]:  


Arguments received:
train_path: /kaggle/working/hackaton/data/C/train.json.gz
test_path: /kaggle/working/hackaton/data/C/test.json.gz
num_checkpoints: 5
device: 1
gnn: gin-virtual
drop_ratio: 0.5
num_layer: 5
emb_dim: 300
batch_size: 32
epochs: 200
baseline_mode: 3
noise_prob: 0.2


In [14]:
import torch.nn.functional as F

class NoisyCrossEntropyLoss(torch.nn.Module):
    def __init__(self, p_noisy):
        super().__init__()
        self.p = p_noisy
        self.ce = torch.nn.CrossEntropyLoss(reduction='none')

    def forward(self, logits, targets):
        losses = self.ce(logits, targets)
        weights = (1 - self.p) + self.p * (1 - torch.nn.functional.one_hot(targets, num_classes=logits.size(1)).float().sum(dim=1))
        return (losses * weights).mean()

class GCELoss(torch.nn.Module):
    def __init__(self, q=0.7):
        super().__init__()
        assert q > 0 and q <= 1, "q must be in (0,1]"
        self.q = q

    def forward(self, logits, labels):
        probs = F.softmax(logits, dim=1).clamp(min=1e-7, max=1.0)
        target_probs = probs[range(len(labels)), labels]
        loss = (1.0 - target_probs.pow(self.q)) / self.q
        return loss.mean()
        
class SCELoss(torch.nn.Module):
    def __init__(self, alpha=0.5, beta=0.5, num_classes=6):
        super().__init__()
        self.alpha = alpha
        self.beta = beta
        self.ce = torch.nn.CrossEntropyLoss()
        self.num_classes = num_classes

    def forward(self, pred, labels):
        ce = self.ce(pred, labels)
        pred_soft = F.softmax(pred, dim=1).clamp(min=1e-7, max=1.0)
        labels_one_hot = F.one_hot(labels, self.num_classes).float()
        rce = -torch.sum(pred_soft * torch.log(labels_one_hot + 1e-7), dim=1).mean()
        return self.alpha * ce + self.beta * rce

In [15]:
script_dir = os.getcwd() 
# device = torch.device(f"cuda:{args.device}" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_checkpoints = args.num_checkpoints if args.num_checkpoints else 3
    
if args.gnn == 'gin':
    model = GNN(gnn_type='gin', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=False).to(device)
elif args.gnn == 'gin-virtual':
    model = GNN(gnn_type='gin', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=True).to(device)
elif args.gnn == 'gcn':
    model = GNN(gnn_type='gcn', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=False).to(device)
elif args.gnn == 'gcn-virtual':
    model = GNN(gnn_type='gcn', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=True).to(device)
else:
    raise ValueError('Invalid GNN type')
    
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
# criterion = torch.nn.CrossEntropyLoss()
if args.baseline_mode == 2:
    criterion = NoisyCrossEntropyLoss(args.noise_prob)
elif args.baseline_mode == 1:
    criterion = torch.nn.CrossEntropyLoss()
elif args.baseline_mode == 3:
    criterion = GCELoss()
else:
    criterion = SCELoss()

In [16]:
test_dir_name = os.path.basename(os.path.dirname(args.test_path))
logs_folder = os.path.join(script_dir, "logs", test_dir_name)
log_file = os.path.join(logs_folder, "training-OUR.log")
os.makedirs(os.path.dirname(log_file), exist_ok=True)
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(message)s')
logging.getLogger().addHandler(logging.StreamHandler())

checkpoint_path_acc = os.path.join(script_dir, "checkpoints", f"model_{test_dir_name}_best_acc.pth")
checkpoint_path_f1 = os.path.join(script_dir, "checkpoints", f"model_{test_dir_name}_best_f1.pth")
checkpoints_folder = os.path.join(script_dir, "checkpoints", test_dir_name)
os.makedirs(checkpoints_folder, exist_ok=True)

In [17]:
if os.path.exists(checkpoint_path_acc) and not args.train_path:
    model.load_state_dict(torch.load(checkpoint_path_acc))
    print(f"Loaded best model from {checkpoint_path_acc}")

In [18]:
log_train = f"logs_our/{test_dir_name}/train.log"
os.makedirs(os.path.dirname(log_train), exist_ok=True)
log_val = f"logs_our/{test_dir_name}/val.log"
os.makedirs(os.path.dirname(log_val), exist_ok=True)

if args.train_path:
    full_dataset = GraphDataset(args.train_path, transform=add_zeros)
    val_size = int(0.2 * len(full_dataset))
    train_size = len(full_dataset) - val_size

    
    generator = torch.Generator().manual_seed(12)
    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size], generator=generator)

    train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False)

    num_epochs = args.epochs
    best_val_accuracy = 0.0   
    best_val_f1 = 0.0

    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []

    if num_checkpoints > 1:
        checkpoint_intervals = [int((i + 1) * num_epochs / num_checkpoints) for i in range(num_checkpoints)]
    else:
        checkpoint_intervals = [num_epochs]

    for epoch in range(num_epochs):
        train_loss, train_acc, train_f1 = train(
            train_loader, model, optimizer, criterion, device,
            save_checkpoints=(epoch + 1 in checkpoint_intervals),
            checkpoint_path=os.path.join(checkpoints_folder, f"model_{test_dir_name}"),
            current_epoch=epoch
        )

        val_loss,val_acc, val_f1 = evaluate(val_loader, model, device, calculate_accuracy=True)

        print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Train F1: {train_f1:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}")
        logging.info(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Train F1: {train_f1:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f},")
        
        with open(log_train, 'a') as f:
            f.write(f"Epoch {epoch + 1}: train_loss: {train_loss}, train_acc: {train_acc}, train_f1: {train_f1}\n")
        
        with open(log_val, 'a') as f:
            f.write(f"Epoch {epoch + 1}: val_loss: {val_loss}, val_acc: {val_acc}, val_f1: {val_f1}\n")
        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        
        if val_acc > best_val_accuracy:
            best_val_accuracy = val_acc
            torch.save(model.state_dict(), checkpoint_path_acc)
            print(f"Best model updated and saved at {checkpoint_path_acc}")
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            torch.save(model.state_dict(), checkpoint_path_f1)
            print(f"Best model updated and saved at {checkpoint_path_f1}")
            

    plot_training_progress(train_losses, train_accuracies, os.path.join(logs_folder, "plots"))
    plot_training_progress(val_losses, val_accuracies, os.path.join(logs_folder, "plotsVal"))

Iterating eval graphs: 100%|██████████| 48/48 [00:06<00:00,  7.33batch/s]
Epoch 1/200, Train Loss: 0.8682, Train Acc: 0.3796, Train F1: 0.1248, Val Loss: 7.0571, Val Acc: 0.3672, Val F1: 0.1188,


Epoch 1/200, Train Loss: 0.8682, Train Acc: 0.3796, Train F1: 0.1248, Val Loss: 7.0571, Val Acc: 0.3672, Val F1: 0.1188
Best model updated and saved at /kaggle/working/hackaton/checkpoints/model_C_best_acc.pth
Best model updated and saved at /kaggle/working/hackaton/checkpoints/model_C_best_f1.pth


Iterating eval graphs: 100%|██████████| 48/48 [00:06<00:00,  7.38batch/s]
Epoch 2/200, Train Loss: 0.8512, Train Acc: 0.3895, Train F1: 0.1122, Val Loss: 3.1271, Val Acc: 0.3913, Val F1: 0.1041,


Epoch 2/200, Train Loss: 0.8512, Train Acc: 0.3895, Train F1: 0.1122, Val Loss: 3.1271, Val Acc: 0.3913, Val F1: 0.1041
Best model updated and saved at /kaggle/working/hackaton/checkpoints/model_C_best_acc.pth


Iterating eval graphs: 100%|██████████| 48/48 [00:06<00:00,  7.29batch/s]
Epoch 3/200, Train Loss: 0.8470, Train Acc: 0.3916, Train F1: 0.1138, Val Loss: 3.7072, Val Acc: 0.3900, Val F1: 0.1095,


...

Epoch 199/200, Train Loss: 0.2864, Train Acc: 0.7977, Train F1: 0.7856, Val Loss: 8.9881, Val Acc: 0.7715, Val F1: 0.7378


Iterating training graphs: 100%|██████████| 192/192 [00:35<00:00,  5.47batch/s]


Checkpoint saved at /kaggle/working/hackaton/checkpoints/C/model_C_epoch_200.pth


Iterating eval graphs: 100%|██████████| 48/48 [00:06<00:00,  7.36batch/s]
Epoch 200/200, Train Loss: 0.2953, Train Acc: 0.7905, Train F1: 0.7765, Val Loss: 5.0813, Val Acc: 0.7591, Val F1: 0.6935,


Epoch 200/200, Train Loss: 0.2953, Train Acc: 0.7905, Train F1: 0.7765, Val Loss: 5.0813, Val Acc: 0.7591, Val F1: 0.6935


In [19]:
import gc
del train_dataset
del train_loader
del full_dataset
del val_dataset
del val_loader
gc.collect()

12373

In [20]:
test_dataset = GraphDataset(args.test_path, transform=add_zeros)
test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False)

In [21]:
model.load_state_dict(torch.load(checkpoint_path_acc))
predictions = evaluate(test_loader, model, device, calculate_accuracy=False)
save_predictions(predictions, args.test_path)

Iterating eval graphs: 100%|██████████| 48/48 [00:06<00:00,  7.32batch/s]

Predictions saved to /kaggle/working/hackaton/submission/testset_C.csv


In [1]:
import shutil
shutil.make_archive('/kaggle/working/logs_our', 'zip', '/kaggle/working/hackaton/logs_our')
shutil.make_archive('/kaggle/working/checkpoints', 'zip', '/kaggle/working/hackaton/checkpoints')

'/kaggle/working/checkpoints.zip'